# 결국엔 완성이 된거 같아서 합체 후 작업
3. requirements.csv -> 1번과 2번을 활용한 요구사항 명세서

In [19]:
import pandas as pd
import openai
import requests
import ast
import slack_alarm as alarm
import json_parsing

In [20]:
# temperature가 높게 작성을 하게 만들어서 약간 오류가 있긴함
## 해당 파일이 수정된 파일
df = pd.read_csv('test.csv')
df.head()

,subject,user_input,requirements
0,"GQ 서비스는 학습 중심의 스터디 그룹 운영을 지원하는 웹 플랫폼으로, 단순한 그룹...","[{'projectName': 'StudyBuddy', 'projectTarget'...","[{'requirementType': 'FUNCTIONAL', 'content': ..."
1,‘기억박물관’은 치매 환자와 그 가족을 위한 따뜻한 기억 회복 웹사이트입니다. 사용...,"[{'projectName': '기억의 다리', 'projectTarget': '치...","[{'requirementType': 'FUNCTIONAL', 'content': ..."
2,"이 앱은 보행 약자들을 위한 지하철 길찾기 앱으로, 계단 없이 이동할 수 있는 경로...","[{'projectName': 'EasyTransit Navigator', 'pro...","[{'requirementType': 'FUNCTIONAL', 'content': ..."
3,"이 앱은 1인 가구를 위한 맞춤형 플랫폼으로, 주로 커뮤니티 기능에 중점을 두고 있...","[{'projectName': '이웃과의 소통 플랫폼', 'projectTarget...","[{'requirementType': 'FUNCTIONAL', 'content': ..."
4,"이 웹앱은 응급실 및 병원 찾기 기능을 제공하여, 응급 상황에서 사용자가 빠르고 안...","[{'projectName': 'EmergencyCare Finder', 'proj...","[{'requirementType': 'FUNCTIONAL', 'content': ..."


## 978개로 데이터 생성 진행
- AWS에 있는거 활용... 을 하면 좀 그런가
- 아무도 안 쓰는거 같으니까 테스트 겸 한번 해보자
- AWS 주소 : http://13.209.5.218:8000/

In [21]:
df.columns

Index(['subject', 'user_input', 'requirements'], dtype='object')

### 진행방법
1. 요구사항 명세서 추가 생성
2. 프로젝트 요약 생성
3. ERD,API 명세서 생성

In [22]:
import re
import logging

base_url = 'http://127.0.0.1:8000/'

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def clean_escaped_quotes(text):
    """이스케이프된 따옴표를 정리하는 함수"""
    
    if not isinstance(text, str):
        return str(text)  # 문자열이 아니면 그대로 반환

    # 방법 1: 정규식으로 \' -> ' 변환
    cleaned = re.sub(r"\\'", "'", text)
    return str(cleaned)

def safe_parse_requirements(text):
    """안전하게 requirements 문자열을 파싱하는 함수"""

     # 이미 딕셔너리나 리스트인 경우 그대로 반환
    if isinstance(text, (dict, list)):
        return str(text)
    
    # 문자열이 아닌 경우 처리
    if not isinstance(text, str):
        logger.warning(f"예상치 못한 타입: {type(text)}")
        return str(text)
    
    try:
        # 1단계: 이스케이프된 따옴표 정리
        cleaned = clean_escaped_quotes(text)
        
        # 2단계: ast.literal_eval로 파싱 시도
        parsed = ast.literal_eval(cleaned)
        
        return str(parsed)
    
    except (ValueError, SyntaxError) as e:
        print(f"ast.literal_eval 실패: {e}")
        
        try:
            # 3단계: 추가 정리 후 재시도
            # 바깥쪽 따옴표 제거 (만약 있다면)
            if cleaned.startswith("'") and cleaned.endswith("'"):
                cleaned = cleaned[1:-1]
            elif cleaned.startswith('"') and cleaned.endswith('"'):
                cleaned = cleaned[1:-1]
            
            # 다시 파싱 시도
            parsed = ast.literal_eval(cleaned)
            return str(parsed)
            
        except Exception as e2:
            print(f"2차 파싱도 실패: {e2}")
            return None
    
# 요구사항 명세서 생성 함수
def generate_requirements(project_overview, requirements) :
    """
    project_overview : 유저가 입력한 project 데이터(json 형태)
    requirements : 요구사항 명세서(json 형태)
    """
    # 데이터 input
    data = {
        "project_overview": project_overview,
        "existing_requirements": requirements,
        "additional_count": 5,
        "max_tokens": 4000,
        "temperature": 0.5,
        "model": "gpt-4o-mini"
    }

    # 데이터 호출
    response = requests.post(
                            url = base_url +'api/PJA/requirements/generate', 
                            json = data,
                            headers={"Content-Type": "application/json"})
    
    new_requirements = response.json()

    # 원본 데이터에 추가로 받은 데이터 추가
    add_requirements = new_requirements['requirements']
    total_requirements = str(ast.literal_eval(requirements) + add_requirements)

    return total_requirements

# 프로젝트 요약 생성 함수 
def generate_project_info(project_overview, total_requirements):
    """
    project_overview : 유저가 입력한 project 데이터(json 형태)
    total_requirements : 요구사항 명세서 생성하는 코드(json 형태)
    """

    # 데이터 input
    data = {
        "project_overview": project_overview,
        "requirements": total_requirements,
        "max_tokens": 4000,
        "temperature": 0.5,
        "model": "gpt-4o-mini"
    }
    
    # 데이터 호출
    response = requests.post(
                            url = base_url +'api/PJA/json_Summury/generate', 
                            json = data,
                            headers={"Content-Type": "application/json"})
    project_info = response.json()

    return str(project_info)

# ERDAPI 생성 함수수
def generate_ERDAPI(project_overview, total_requirements, project_info) :
    """
    project_overview : 유저가 입력한 project 데이터(json 형태)
    total_requirements : 요구사항 명세서 생성하는 데이터(json 형태)
    project_info : 프로젝트 요약한 데이터(json 형태태)
    """

    # 데이터 input
    data = {
            "project_overview": project_overview,
            "requirements": total_requirements,
            "project_summury": project_info,
            "max_tokens": 6000,
            "temperature": 0.5,
            "model": "gpt-4o-mini"
        }
    
    # 데이터 호출
    response = requests.post(
                        url = base_url +'api/PJA/json_ERDAPI/generate', 
                        json = data,
                        headers={"Content-Type": "application/json"})

    if response.status_code == 200 :
        ERDAPI_data = response.json()
        return str(ERDAPI_data)
    else : 
        print({response.status_code})
        return None

In [27]:
import time

# 테스트로 10개만
project_overview_lst = df['user_input'][:5]
requirements_lst= df['requirements'][:5]

# 더미로 돌렸을때 걸리는 시간 체크도 필요할듯

# 데이터 저장해놓을 리스트
total_requirements_lst, project_info_lst, ERDAPI_data_lst = [], [], []

for project_overview, requirements, num in zip(project_overview_lst, requirements_lst, range(len(requirements_lst))) :

    # 각 코드 실행 후 리스트에 백업
    total_requirements = generate_requirements(project_overview, requirements)
    total_requirements = json_parsing.extract_json_from_response(total_requirements)
    total_requirements_lst.append(total_requirements)

    project_info = generate_project_info(project_overview, total_requirements)
    project_info = json_parsing.extract_json_from_response(project_info)
    project_info_lst.append(project_info)

    ERDAPI_data = generate_ERDAPI(project_overview, total_requirements, project_info)
    ERDAPI_data = json_parsing.extract_json_from_response(ERDAPI_data)
    ERDAPI_data_lst.append(ERDAPI_data)

    if num+1 % 50 == 0 :
        alarm.send_slack(f"{num}번째 실행 완료")

    time.sleep(1)

alarm.send_slack(f"더미데이터 생성 작업 완료")

True

In [13]:
total_requirements_lst

["[{'requirementType': 'FUNCTIONAL', 'content': '사용자는 회원가입 시 이메일 인증을 통해 계정을 활성화할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '사용자는 스터디 그룹 내에서 퀴즈를 생성하고, 다른 사용자가 생성한 퀴즈에 참여할 수 있어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '공지사항 게시판은 그룹 내 모든 회원이 읽고 댓글을 남길 수 있어야 하며, 관리자는 공지사항을 삭제할 수 있어야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '시스템은 사용자 요청에 대해 평균 2초 이내에 응답해야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '시스템은 동시 사용자 100명 이상을 지원할 수 있어야 하며, 이 경우에도 응답 시간이 3초를 초과하지 않아야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '데이터베이스는 1,000개의 스터디 그룹과 각 그룹당 최대 500개의 퀴즈를 처리할 수 있어야 하며, 데이터 조회는 1초 이내에 완료되어야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '사용자는 스터디 그룹의 과제를 생성하고, 다른 사용자들이 과제를 제출할 수 있도록 해야 한다.'}, {'requirementType': 'FUNCTIONAL', 'content': '사용자는 자신의 퀴즈 결과를 확인하고, 이전 퀴즈에 대한 피드백을 받을 수 있어야 한다.'}, {'requirementType': 'PERFORMANCE', 'content': '시스템은 평균 1000개의 퀴즈를 동시에 생성할 수 있어야 하며, 이 경우에도 퀴즈 생성 요청에 대한 응답 시간이 4초를 초과하지 않아야 한다.'}, {'requirementType': 

In [14]:
project_info_lst

["{'json': {'project_info': {'title': 'StudyBuddy', 'category': '학습 관리 플랫폼', 'target_users': ['대학생', '자격증 준비생'], 'core_features': ['스터디 그룹 생성 및 관리', '퀴즈 생성 및 참여 기능', '공지사항 및 과제 게시판'], 'technology_stack': ['Java Servlet', 'JSP', 'MySQL'], 'problem_solving': {'current_problem': '효율적인 학습을 위한 스터디 그룹 운영의 어려움', 'solution_idea': '스터디 그룹 내에서 학습 내용을 복습하고 소통할 수 있는 기능 제공', 'expected_benefits': ['사용자 간의 소통 강화', '학습 동기 부여', '효율적인 학습 관리']}}}, 'model': 'gpt-4o-mini', 'total_tokens': 1624, 'prompt_tokens': 1443, 'completion_tokens': 181}",
 "{'json': {'project_info': {'title': '기억의 다리', 'category': '치매 환자 지원', 'target_users': ['치매 환자', '가족'], 'core_features': ['일상 사진 업로드 및 기억 회상 기능', 'AI 챗봇과의 대화를 통한 정보 되새김', '디지털 다이어리 관리 및 가족 소통 기능'], 'technology_stack': ['Python (Django)', 'React', 'PostgreSQL'], 'problem_solving': {'current_problem': '치매 환자의 기억 회복 및 가족 간의 소통 부족', 'solution_idea': '일상 사진과 AI 챗봇을 활용하여 기억을 되새기고 가족과 소통할 수 있는 플랫폼 제공', 'expected_benefits': ['치매 환자의 삶의 질 향상', '가족 간의 정서적 유대감 강화']}}}, 'model

In [15]:
ERDAPI_data_lst

["{'json': {'erd_tables': [{'name': 'users', 'erd_columns': [{'name': 'userId', 'data_type': 'integer', 'is_primary_key': True, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'username', 'data_type': 'string', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'email', 'data_type': 'string', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'password', 'data_type': 'string', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'createdAt', 'data_type': 'datetime', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}]}, {'name': 'study_groups', 'erd_columns': [{'name': 'groupId', 'data_type': 'integer', 'is_primary_key': True, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'groupName', 'data_type': 'string', 'is_primary_key': False, 'is_foreign_key': False, 'is_nullable': False}, {'name': 'createdBy', 'data_type': 'integer', 'is_primary_key': False, 'is_fore